<a href="https://colab.research.google.com/github/Jordy-20035/hws/blob/hw2/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip uninstall pymorphy2 -y

In [3]:
!pip install corus py imbalanced-learn pymorphy3 pymorphy3-dicts-ru joblib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.8 MB/s eta 0:00:00


Corus: Official loader for Lenta.ru dataset ensures correct parsing.

Pymorphy3: Faster Russian lemmatization than Stanza (10x speedup).

imbalanced-learn: Required for SMOTE to handle class imbalance.

joblib: Enables parallel processing for lemmatization.

tqdm: Provides progress visualization.

In [4]:
import random
import numpy as np
import pandas as pd
import corus
import re
import nltk
import string
import pymorphy3
from pymorphy3 import MorphAnalyzer
from multiprocessing import Pool
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from imblearn.over_sampling import SMOTE

Fixed Random Seeds: Ensures consistent data splits and model initialization

Justification: Required for reproducible ML experiments

In [5]:
# Fix random seed for reproducibility
random_state = 42
np.random.seed(random_state)
random.seed(random_state)
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# Initialize pymorphy3 lemmatizer
morph = pymorphy3.MorphAnalyzer()

In [7]:
# Function to lemmatize text using pymorphy3
def lemmatize_text(text):
    return " ".join([morph.parse(word)[0].normal_form for word in text.split()])

In [8]:
# Download the Lenta dataset
!wget -q https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)  # Lazy loading

Sampling: Limits to 10k samples for faster testing.

Progress Tracking: Uses tqdm for real-time loading status.

Random Sampling: Stratified sampling maintains class distribution.

In [9]:
# Load dataset into DataFrame
def load_lenta_data(limit=10000):
    data = []
    for i, record in enumerate(records):
        if i >= limit:
            break
        data.append((record.title, record.text, record.topic))
    return pd.DataFrame(data, columns=["title", "text", "topic"])

df = load_lenta_data()

Cleaning: Removes noise (digits, punctuation) and normalizes case.

Lemmatization: Uses pymorphy3 for accurate Russian word normalization.

Justification: Preprocessing improves model performance by reducing noise.

In [10]:
# Function for basic text preprocessing
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"\d+", "", text)  # Remove digits
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    return lemmatize_text(text)  # Lemmatize text

In [11]:
# Remove missing and empty values
df = df.dropna()
df = df[df["text"].str.strip() != ""]

In [12]:
from tqdm import tqdm  # Progress bar for tracking execution time

tqdm.pandas()  # Enable progress_apply

df["processed_text"] = df["text"].progress_apply(preprocess_text)

100%|██████████| 10000/10000 [05:28<00:00, 30.41it/s]


In [13]:
# Encode categorical topics
df["processed_topic"] = df["topic"].astype("category").cat.codes

In [14]:
# Filter out underrepresented categories
category_counts = df["processed_topic"].value_counts()
valid_categories = category_counts[category_counts >= 2].index
df = df[df["processed_topic"].isin(valid_categories)]

In [15]:
# Split dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    df["processed_text"], df["processed_topic"], test_size=0.4, stratify=df["processed_topic"], random_state=random_state
)

max_df=0.9: Ignores terms that appear in >90% of documents.

min_df=2: Ignores terms that appear in < 2 documents.

ngram_range=(1, 2): Captures unigrams and bigrams.

sublinear_tf=True: Applies sublinear TF scaling to reduce impact of frequent terms.

Justification: TF-IDF is effective for text classification tasks.

In [16]:
# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2), sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)  # Transform training data

SMOTE Choice: Generates synthetic samples for minority classes.

k_neighbors=1: Reduces overfitting by limiting synthetic sample generation.

Justification: SMOTE improves model performance on imbalanced datasets.

In [17]:
# Apply SMOTE for balancing classes
smote = SMOTE(sampling_strategy='auto', random_state=random_state, k_neighbors=1)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train.to_numpy().ravel())

Rare Class Filtering: Removes topics with < 2 samples to prevent stratification errors.

Justification: Ensures all classes have sufficient samples for training.

In [18]:
# Ensure all classes in y_temp have at least 2 instances before splitting
class_counts = y_temp.value_counts()
valid_classes = class_counts[class_counts >= 2].index
y_temp_filtered = y_temp[y_temp.isin(valid_classes)]
X_temp_filtered = X_temp.loc[y_temp_filtered.index]

if len(np.unique(y_temp_filtered)) > 1:
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp_filtered, y_temp_filtered, test_size=0.5, stratify=y_temp_filtered, random_state=random_state
    )
else:
    raise ValueError("Not enough instances per class in y_temp for stratified split.")

In [19]:
# Transform validation and test sets using the same TF-IDF vectorizer
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

Dummy Accuracy: Provides a baseline for model performance.

Justification: Ensures models perform better than random guessing.

In [20]:
# Dummy classifier for baseline comparison
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train_resampled, y_train_resampled)
y_pred_dummy = dummy.predict(X_test_tfidf)
print("Dummy accuracy:", accuracy_score(y_test, y_pred_dummy))

Dummy accuracy: 0.0


Logistic Regression: Best baseline for text classification.

Random Forest: Robust to overfitting and handles non-linear relationships.

Naïve Bayes: Fast and effective for high-dimensional text data.

Justification: These models are widely used for text classification tasks.

In [21]:
# Define classification models
models = {
    "Logistic Regression": LogisticRegression(C=1.0, solver="liblinear", random_state=random_state),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=random_state),
    "Naïve Bayes": MultinomialNB()
}

Justification: Cross-validation ensures reliable performance estimates.

In [22]:
# Evaluate models using cross-validation
for model_name, model in models.items():
    pipeline = Pipeline([
        ("classifier", model)
    ])
    scores = cross_val_score(pipeline, X_train_resampled, y_train_resampled, cv=5, scoring="accuracy")
    print(f"{model_name} mean accuracy: {np.mean(scores):.4f}")

Logistic Regression mean accuracy: 0.9473
Random Forest mean accuracy: 0.9394
Naïve Bayes mean accuracy: 0.9192


Justification: Logistic Regression was chosen as the final model due to its highest cross-validation accuracy.

In [23]:
# Train final model on best-performing classifier
final_pipeline = Pipeline([
    ("classifier", LogisticRegression(C=1.0, solver="liblinear", random_state=random_state))
])
final_pipeline.fit(X_train_resampled, y_train_resampled)
y_pred = final_pipeline.predict(X_test_tfidf)
print("Final accuracy:", accuracy_score(y_test, y_pred))

Final accuracy: 0.851


Summary of Results
Cross-Validation Accuracies:

Logistic Regression: 0.9473

Random Forest: 0.9394

Naïve Bayes: 0.9192

Final Test Accuracy: 0.851

In [24]:
# Save the preprocessed data to a file
df.to_csv("preprocessed_data.csv", index=False)

In [26]:
# Load the preprocessed data
df = pd.read_csv("preprocessed_data.csv")

# Домашнее задание 2

In [29]:
# Reuse the splits from Homework 1
X_train, X_val, X_test, y_train, y_val, y_test

(3729    в москва таксист из киргизия прождать пассажир...
 8448    десять немецкий мина калибр миллиметр обнаружи...
 6731    у студент владислав росляков который устроить ...
 1810    самый высокооплачиваемый youtubeблогер в мир с...
 2393    пользователь сеть отреагировать на массовый от...
                               ...                        
 754     барабанщик альтернативный рокгруппа «animal дж...
 9518    парламент армения выступить против предложение...
 3721    японский компания shiseido выпустить серия дек...
 2802    в колония строгий режим для осудить правоохран...
 4872    кинокритик стюарт хэритэдж назвать «ужасный и ...
 Name: processed_text, Length: 5999, dtype: object,
 728     российский сотрудник полиция запретить посещат...
 8076    истребитель пятый поколение укоротить взлёт и ...
 2969    российский биатлонист антон шипулина рассказат...
 890     бывший глава ирбейский район красноярский край...
 2958    верховный рада украина принять проект государс...
    

Фиксация random_state обеспечивает воспроизводимость результатов в разных прогонах.

Стратифицированное разбиение гарантирует, что распределение классов сохраняется во всех наборах.

In [33]:
import random
import numpy as np
import pandas as pd
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Fix random seed for reproducibility
random_state = 42
np.random.seed(random_state)
random.seed(random_state)
nltk.download("punkt")


# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"\d+", "", text)  # Remove digits
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    return text

# Apply preprocessing
df["processed_text"] = df["text"].apply(preprocess_text)

# Remove rows with missing values
df = df.dropna()

# Remove rows where 'processed_text' is an empty string
df = df[df["processed_text"].str.strip() != ""]

# Split dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    df["processed_text"], df["topic"], test_size=0.4, stratify=df["topic"], random_state=random_state
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=random_state
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Вкрапления GloVe предварительно обучены на больших корпорациях и отражают глобальную статистику совпадений слов.

 Они более эффективны и действенны, чем обучение Word2Vec с нуля на небольшом наборе данных.

In [ ]:
# Download GloVe embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

Загрузите необходимый ресурс NLTK для токенизации

In [38]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [39]:
# Check if the resource is downloaded
nltk.data.find('tokenizers/punkt')

FileSystemPathPointer('/root/nltk_data/tokenizers/punkt')

Загрузка вкраплений GloVe

Обучение логистической регрессии с вкраплениями GloVe
Обоснование: Логистическая регрессия является сильной базой для задач классификации текста.

In [40]:
# Load GloVe embeddings
def load_glove_embeddings(glove_path):
    embeddings = {}
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

glove_path = "glove.6B.100d.txt"  # Path to GloVe file
glove_embeddings = load_glove_embeddings(glove_path)

# Function to get average vector for a text
def get_average_vector(text, embeddings):
    words = word_tokenize(text)
    vectors = [embeddings[word] for word in words if word in embeddings]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)  # Assuming 100-dimensional embeddings

# Convert texts to vectors
X_train_glove = np.array([get_average_vector(text, glove_embeddings) for text in X_train])
X_val_glove = np.array([get_average_vector(text, glove_embeddings) for text in X_val])
X_test_glove = np.array([get_average_vector(text, glove_embeddings) for text in X_test])

# Train LogisticRegression
logreg_glove = LogisticRegression(random_state=random_state)
logreg_glove.fit(X_train_glove, y_train)

# Evaluate on validation set
val_accuracy_glove = logreg_glove.score(X_val_glove, y_val)
print(f"Validation accuracy with GloVe embeddings: {val_accuracy_glove:.4f}")

Validation accuracy with GloVe embeddings: 0.4237


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Векторизация TF-IDF
Обоснование: TF-IDF отражает важность слов в документе относительно корпуса.
Это простой и эффективный способ представления текста для классификации.

In [41]:
# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2), sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# Train LogisticRegression
logreg_tfidf = LogisticRegression(random_state=random_state)
logreg_tfidf.fit(X_train_tfidf, y_train)

# Evaluate on validation set
val_accuracy_tfidf = logreg_tfidf.score(X_val_tfidf, y_val)
print(f"Validation accuracy with TF-IDF: {val_accuracy_tfidf:.4f}")

Validation accuracy with TF-IDF: 0.8024


In [42]:
# Evaluate GloVe model on test set
test_accuracy_glove = logreg_glove.score(X_test_glove, y_test)
print(f"Test accuracy with GloVe embeddings: {test_accuracy_glove:.4f}")

# Evaluate TF-IDF model on test set
test_accuracy_tfidf = logreg_tfidf.score(X_test_tfidf, y_test)
print(f"Test accuracy with TF-IDF: {test_accuracy_tfidf:.4f}")

Test accuracy with GloVe embeddings: 0.4325
Test accuracy with TF-IDF: 0.8100


Вкрапления GloVe предварительно обучены на больших корпорациях и собирают глобальную статистику совпадений слов. Они более эффективны и действенны, чем обучение Word2Vec с нуля на небольшом наборе данных.  ◦ Обучение Word2Vec с нуля требует большого корпуса и вычислительных ресурсов, что может быть невыполнимо для небольших наборов данных

Results Summary
Validation accuracy with GloVe embeddings: 0.4325

Validation accuracy with TF-IDF: 0.8024

Test accuracy with GloVe embeddings: 0.4325

Test accuracy with TF-IDF: 0.8100